***starting spark session***

In [8]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=87d984fb2d0446300a90500ef1272c67fe5495c8a85494288747724cb62ff5c8
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.appName("example").config("spark.some.config.option","some-value").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/03 14:12:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


***collecting dataset***

In [13]:
df_raw=spark.read.format('com.databricks.spark.csv').options(header='true',inferschema='true').load('/kaggle/input/training/online_retail_II.csv',header=True)

In [14]:
df_raw.show(5)

+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|     InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+
| 536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26| 2.55|      17850|United Kingdom|
| 536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|
| 536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 08:26| 2.75|      17850|United Kingdom|
| 536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|
| 536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|
+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+
only showing top 5 rows



***data preprocessing***

In [16]:
df_raw.printSchema()

root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Customer ID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [17]:
from pyspark.sql.functions import count
def my_count(df_in):
    df_in.agg( *[ count(c).alias(c) for c in df_in.columns ]).show()
my_count(df_raw)

+-------+---------+-----------+--------+-----------+------+-----------+-------+
|Invoice|StockCode|Description|Quantity|InvoiceDate| Price|Customer ID|Country|
+-------+---------+-----------+--------+-----------+------+-----------+-------+
| 541910|   541910|     540456|  541910|     541910|541910|     406830| 541910|
+-------+---------+-----------+--------+-----------+------+-----------+-------+



In [18]:
df=df_raw.dropna(how='any')
my_count(df)

+-------+---------+-----------+--------+-----------+------+-----------+-------+
|Invoice|StockCode|Description|Quantity|InvoiceDate| Price|Customer ID|Country|
+-------+---------+-----------+--------+-----------+------+-----------+-------+
| 406830|   406830|     406830|  406830|     406830|406830|     406830| 406830|
+-------+---------+-----------+--------+-----------+------+-----------+-------+



In [22]:
from pyspark.sql.functions import to_utc_timestamp, unix_timestamp, lit,datediff,col
timeFmt="dd-MM-yyyy HH:mm"
df=df.withColumn('NewInvoiceDate', to_utc_timestamp(unix_timestamp(col('InvoiceDate'),timeFmt).cast('timestamp'),'UTC'))
df.show(5)

+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+-------------------+
|Invoice|StockCode|         Description|Quantity|     InvoiceDate|Price|Customer ID|       Country|     NewInvoiceDate|
+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+-------------------+
| 536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26| 2.55|      17850|United Kingdom|2010-12-01 08:26:00|
| 536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|2010-12-01 08:26:00|
| 536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 08:26| 2.75|      17850|United Kingdom|2010-12-01 08:26:00|
| 536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|2010-12-01 08:26:00|
| 536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|2010-12-01 08:26:00|
+-------+---------+--------------------+

In [44]:
from pyspark.sql.functions import round
df=df.withColumn('TotalPrice',round(df.Quantity * df.Price, 2))
from pyspark.sql.functions import mean,min,max,sum,datediff,to_date
date_max=df.select(max('NewInvoiceDate')).toPandas()
current=to_utc_timestamp(unix_timestamp(lit(str(date_max.iloc[0][0])),'yyyy-MM-dd HH:mm').cast('timestamp'),'UTC')

In [45]:
df=df.withColumn('Duration', datediff(lit(current), 'NewInvoiceDate'))
df.show(5)


+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+-------------------+----------+--------+
|Invoice|StockCode|         Description|Quantity|     InvoiceDate|Price|Customer ID|       Country|     NewInvoiceDate|TotalPrice|Duration|
+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+-------------------+----------+--------+
| 536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26| 2.55|      17850|United Kingdom|2010-12-01 08:26:00|      15.3|     373|
| 536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|2010-12-01 08:26:00|     20.34|     373|
| 536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 08:26| 2.75|      17850|United Kingdom|2010-12-01 08:26:00|      22.0|     373|
| 536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|2010-12-01 08:26:00|     20.34|     373|
| 536365|   84029E|R

*****RFM-ANALYSIS*****

In [46]:
from pyspark.sql.functions import datediff, lit, current_timestamp
recency = df.groupBy('Customer ID').agg(min('Duration').alias('Recency'))
frequency=df.groupBy('Customer ID','Invoice').count().groupBy('Customer ID').agg(count("*").alias("Frequency"))
monetary=df.groupBy('Customer ID').agg(round(sum('TotalPrice'),2).alias("Monetary"))
rfm = recency.join(frequency, 'Customer ID', how='inner').join(monetary, 'Customer ID', how='inner')
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
rfm.show(5)

+-----------+-------+---------+--------+
|Customer ID|Recency|Frequency|Monetary|
+-----------+-------+---------+--------+
|      17389|      0|       43|31300.08|
|      13623|     30|        7|  672.44|
|      15727|     16|        7| 5178.96|
|      13285|     23|        4| 2709.12|
|      14570|    280|        2|  218.06|
+-----------+-------+---------+--------+
only showing top 5 rows



In [51]:
cols=['Recency','Frequency','Monetary']
import pandas as pd
import numpy as np
rfm_pandas = rfm.toPandas()

# Use the describe method in pandas
description = rfm_pandas[cols].describe()

# Display the description
print(description)

           Recency    Frequency       Monetary
count  4372.000000  4372.000000    4372.000000
mean     91.581199     5.075480    1898.463817
std     100.772139     9.338754    8219.344627
min       0.000000     1.000000   -4287.630000
25%      16.000000     1.000000     293.362500
50%      50.000000     3.000000     648.075000
75%     143.000000     5.000000    1611.725000
max     373.000000   248.000000  279489.020000


In [52]:
def RScore(x):
    if x<=16:
        return 1
    elif x<=50:
        return 2
    elif x<=143:
        return 3
    else:
        return 4
def FScore(x):
    if x<=1:
        return 4
    elif x<=3:
        return 3
    elif x<=5:
        return 2
    else:
        return 1
def MScore(x):
    if x<=293:
        return 4
    elif x<=648:
        return 3
    elif x<=1611:
        return 2
    else:
        return 1
    

In [53]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType,DoubleType
R_udf=udf(lambda x: RScore(x), StringType())
F_udf=udf(lambda x: FScore(x), StringType())
M_udf=udf(lambda x: MScore(x), StringType())
rfm_seg=rfm.withColumn("r_seg", R_udf("Recency"))
rfm_seg=rfm_seg.withColumn("f_seg", F_udf("Frequency"))
rfm_seg=rfm_seg.withColumn("m_seg", M_udf("Monetary"))
rfm_seg.show(5)

+-----------+-------+---------+--------+-----+-----+-----+
|Customer ID|Recency|Frequency|Monetary|r_seg|f_seg|m_seg|
+-----------+-------+---------+--------+-----+-----+-----+
|      17389|      0|       43|31300.08|    1|    1|    1|
|      13623|     30|        7|  672.44|    2|    1|    2|
|      15727|     16|        7| 5178.96|    1|    1|    1|
|      13285|     23|        4| 2709.12|    2|    2|    1|
|      14570|    280|        2|  218.06|    4|    3|    4|
+-----------+-------+---------+--------+-----+-----+-----+
only showing top 5 rows



In [54]:
import pyspark.sql.functions as F
rfm_seg=rfm_seg.withColumn('RFMScore',F.concat(F.col('r_seg'),F.col('f_seg'),F.col('m_seg')))
rfm_seg.sort(F.col('RFMScore')).show(5)

+-----------+-------+---------+--------+-----+-----+-----+--------+
|Customer ID|Recency|Frequency|Monetary|r_seg|f_seg|m_seg|RFMScore|
+-----------+-------+---------+--------+-----+-----+-----+--------+
|      18161|     10|        6| 1612.79|    1|    1|    1|     111|
|      15727|     16|        7| 5178.96|    1|    1|    1|     111|
|      12471|      2|       49|18740.92|    1|    1|    1|     111|
|      17389|      0|       43|31300.08|    1|    1|    1|     111|
|      17809|     16|       15| 4627.62|    1|    1|    1|     111|
+-----------+-------+---------+--------+-----+-----+-----+--------+
only showing top 5 rows



In [55]:
rfm_seg.groupBy('RFMScore').agg({'Recency':'mean','Frequency':'mean','Monetary':'mean'}).sort(F.col('RFMScore')).show(5)

+--------+-----------------+------------------+------------------+
|RFMScore|     avg(Recency)|     avg(Monetary)|    avg(Frequency)|
+--------+-----------------+------------------+------------------+
|     111|6.035123966942149| 8828.888595041319|18.882231404958677|
|     112|7.237113402061856|1223.3604123711339| 7.752577319587629|
|     113|              8.0|          505.9775|               7.5|
|     114|             11.0|            191.17|               8.0|
|     121|6.472727272727273|2569.0619999999994| 4.636363636363637|
+--------+-----------------+------------------+------------------+
only showing top 5 rows

